In [210]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [211]:
print_to_excel = True

In [212]:
# сбор данных из ручного источника с маппингом
file_name = 'Brazil_accounts_full_cover.xlsx'
data_excel = pd.ExcelFile(file_name)
sheet_list = data_excel.sheet_names
data = pd.DataFrame()
for sheet in sheet_list:
    data_comp = pd.read_excel(file_name, sheet_name=sheet)
    if sheet=='BRFTO':
        data_comp.columns = ['glAccount','dis','code_4','bscCode','bcsAccount']
    else:
        data_comp.columns = ['glAccount','dis','bscCode','bcsAccount']
    data_comp = data_comp[~data_comp.bcsAccount.isna()]
    data_comp['buCode'] = sheet
    data = pd.concat([data, data_comp], axis=0)
data.shape

(178, 6)

In [213]:
data_glAccount_list = data.glAccount.dropna().unique().tolist() # mb glAccount
data_bscCode_list = data.bscCode.dropna().unique().tolist() # bscCode
data_bcsAccount_list = data.bcsAccount.dropna().unique().tolist() # bcsAccount
data_code_4_list = data.code_4.dropna().unique().tolist() # glAccount

In [214]:
# # выгружаем полные данные для проверки вхождени столбцов
# data_ARAP_full = export_from_RISKCUSTOM(f'''
# SELECT distinct *
# from RISKACCESS."unifiedARReport" 
# where "reportDate" = TO_DATE('2024-08-31', 'YYYY-MM-DD')
# --and "companyCode" in ({str(sheet_list)[1:-1]})
# fetch first 50000 rows only 
# ''')

# # проверка вхождения столбцов
# s = (data_ARAP_full.isin(data_glAccount_list)).any(axis=0)
# s[s==True]

In [215]:
# выгрузка данных из ARAP
query = f'''
SELECT *
--"companyCode","glAccount","bscCode","bcsAccount","currencyCode","presentAmountUSD"
            from RISKACCESS."unifiedARReport" 
            where "reportDate" = TO_DATE('2024-08-31', 'YYYY-MM-DD')
AND "companyCode" in ({str(sheet_list)[1:-1]})
AND "rowType" = 'M'
AND (
    "bscCode" in ({str(data_bscCode_list)[1:-1]}) 
    OR "bcsAccount" in ({str(data_bcsAccount_list)[1:-1]})
    OR "glAccount" in ({str(data_glAccount_list)[1:-1]})  
    OR "glAccount" in ({str(data_code_4_list)[1:-1]})  
    )
'''
data_ARAP = export_from_RISKCUSTOM(query)
data_ARAP.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(567, 166)

In [217]:
# группировка по строкам ARAP
data_ARAP_cols_list = ['companyCode','glAccount','bscCode','bcsAccount','currencyCode']
data_ARAP_pivot = data_ARAP.groupby(as_index=False, dropna=False, by=data_ARAP_cols_list)['presentAmountUSD'].agg('sum')
# группировка по строкам ручного источника full cover
data_cols_list = ['buCode','glAccount','code_4','bscCode','bcsAccount', 'dis']
data_for_merge = data[data_cols_list].drop_duplicates().reset_index(drop=True)

# merge 
merge_dict = {'glAccount':'glAccount', 'bscCode':'bscCode', 'bcsAccount':'bcsAccount'}
# , 'code_4':'glAccount' - не попали в выборку
# bucode + glAccount on glAccount
data_for_merge.glAccount, data_ARAP_pivot.glAccount = data_for_merge.glAccount.astype(int), data_ARAP_pivot.glAccount.astype(int)
# bucode + glAccount on code_4
data_for_merge.code_4 = data_for_merge.code_4.fillna(0).astype(int)
# bucode + bscCode
data_for_merge.bscCode, data_ARAP_pivot.bscCode = data_for_merge.bscCode.astype(str).replace(' ',''), data_ARAP_pivot.bscCode.astype(str).replace(' ','')
# bucode + bcsAccount
data_for_merge.bcsAccount, data_ARAP_pivot.bcsAccount = data_for_merge.bcsAccount.astype(str).replace(' ',''), data_ARAP_pivot.bcsAccount.astype(str).replace(' ','')
# merge по каждому из столбуов и слияние
full_data_merge = pd.DataFrame()
for key in list(merge_dict.keys()):
    left_on = ['buCode'] + [key]
    right_on = ['companyCode'] + [key]
    data_merge = data_for_merge.merge(data_ARAP_pivot, how='inner', left_on=left_on, right_on=right_on)
    data_merge['merge_on'] = key
    if len(data_merge) != 0:
        full_data_merge = pd.concat([full_data_merge,data_merge])
full_data_merge.head(2)

,buCode,glAccount,code_4,bscCode_x,bcsAccount_x,dis,companyCode,bscCode_y,bcsAccount_y,currencyCode,presentAmountUSD,merge_on,glAccount_x,bscCode,glAccount_y
0,BRHRG,4.500000e+05,0,BSC350,B661000,Emprest Financ - Capital de Giro,BRHRG,nan,B769910,BRL,-3.541349e+06,glAccount,NaN,NaN,NaN
1,BRHRG,4.602100e+05,0,BSC350,B661000,Empréstimos Finimp,BRHRG,nan,B769910,BRL,0.000000e+00,glAccount,NaN,NaN,NaN
2,BRHRG,2.150180e+05,0,BSC350,B661000,(-)Dup desc ATLANTA,BRHRG,nan,B769910,BRL,1.136868e-13,glAccount,NaN,NaN,NaN
3,BRHRG,2.150280e+05,0,BSC350,B661000,(-) Dupl desc Brasil S/A,BRHRG,nan,B769910,BRL,-1.152540e+06,glAccount,NaN,NaN,NaN
4,BRHRG,6.220100e+05,0,BSC400,B673100,Empréstimos Extraconcursal,BRHRG,nan,B769910,BRL,1.000000e-02,glAccount,NaN,NaN,NaN
5,BRHRG,6.220120e+05,0,BSC400,B673100,(-) AVJ - Empréstimos Extraconcursal,BRHRG,nan,B769910,BRL,0.000000e+00,glAccount,NaN,NaN,NaN
6,BRHRG,6.300020e+05,0,BSC400,B673100,Emprest Financ Nacionais - LP Classe II,BRHRG,nan,B769910,BRL,-8.672651e+06,glAccount,NaN,NaN,NaN
7,BRHRG,6.310020e+05,0,BSC400,B673100,Emprest Financ Exterior - LP Classe II,BRHRG,nan,B769910,BRL,-1.980325e+06,glAccount,NaN,NaN,NaN
8,BRHRG,6.310020e+05,0,BSC400,B673100,Emprest Financ Exterior - LP Classe II,BRHRG,nan,B769910,USD,-3.658842e+07,glAccount,NaN,NaN,NaN
9,BRHRG,6.200030e+05,0,BSC400,B673100,Emprest Financ Nacionais - LP Classe III,BRHRG,nan,B769910,BRL,0.000000e+00,glAccount,NaN,NaN,NaN


Описание на англ:

In [219]:
EN_listdir = os.listdir('EN') # watch folder
EN_listdir_sheets = {x:pd.ExcelFile(f'EN\\{x}').sheet_names for x in EN_listdir} # watch all sheets
right_sheets = ['Balance Sheet', 'Comentários Maksim', 'DE-PARA Salitre'] # choose right sheets
EN_listdir_right_sheets = {x:y for x,y in zip(EN_listdir,right_sheets) if y in EN_listdir_sheets[x]} # create dict with rifht sheets
# read all info to one df
EN_df = pd.DataFrame()
for file in EN_listdir:
    file_data = pd.read_excel(f'EN\\{file}', sheet_name=EN_listdir_right_sheets[file])
    if 'Descrição2' in file_data.columns.tolist():
        file_data = file_data.rename(columns={'Descrição2':'Descr Conta', 'Nº conta':'Seg Conta', 'Class1':'Class Account','Class2':'Class Account 2'})
    EN_df = pd.concat([EN_df, file_data])

In [220]:
assert len(EN_df[EN_df['Descr Conta'].isna()]) == 0 # desc is not na
EN_df_work = EN_df[['Seg Conta', 'Descr Conta', 'Account Description (English)', 'Class Account','Class Account 2','Class3']]
EN_df_work.head(2)

,Seg Conta,Descr Conta,Account Description (English),Class Account,Class Account 2,Class3
0,2105010003,Adiantamento de Clientes,Customer Advances,Liabilities,Current Liabilities,NaN
1,2106010001,Vendas para Entrega Futura,Sales for Future Delivery,Liabilities,Current Liabilities,NaN


In [221]:
# merge desc on EN with part wich are in full cover and in ARAP
full_data_merge = full_data_merge.reset_index(drop=True).merge(EN_df_work.reset_index(drop=True), how='left', left_on='dis', right_on='Descr Conta')

Mapping check:

In [222]:
# create conditions df from excel files with final counts
files_df = pd.DataFrame({'file_name':[], 'Balance_Name':[], 'Currency_Denomination':[], 'Class_Account_2':[], 'file_sheet':[], 'Saldo_Final':[]})
n = 0
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRSFT_FX_Exposure_Model_01.10.2024_-_With_Pnl___Tax_Credit.xlsx', 'Balance_Name':'Related parties - loan agreement', 'Currency_Denomination':'BRL, out of OCP', 'Class_Account_2':'Current Liabilities', 'file_sheet':'Balance Sheet', 'Saldo_Final':3457110*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax_Credit.xlsx', 'Balance_Name':'Borrowings Third Parties', 'Currency_Denomination':'USD denominated', 'Class_Account_2':'Current Liabilities', 'file_sheet':'Balance Sheet', 'Saldo_Final':10118*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax_Credit.xlsx', 'Balance_Name':'Borrowings Third Parties', 'Currency_Denomination':'BRL denominated, in OCP', 'Class_Account_2':'Current Liabilities', 'file_sheet':'Balance Sheet', 'Saldo_Final':29996*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax_Credit.xlsx', 'Balance_Name':'Borrowings Third Parties', 'Currency_Denomination':'USD denominated', 'Class_Account_2':'Non-Current Liabilities', 'file_sheet':'Balance Sheet', 'Saldo_Final':162287*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax_Credit.xlsx', 'Balance_Name':'Borrowings Third Parties', 'Currency_Denomination':'BRL denominated, in OCP', 'Class_Account_2':'Non-Current Liabilities', 'file_sheet':'Balance Sheet', 'Saldo_Final':168219*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax_Credit.xlsx', 'Balance_Name':'Borrowings Third Parties', 'Currency_Denomination':'BRL, out of OCP', 'Class_Account_2':'Non-Current Liabilities', 'file_sheet':'Balance Sheet', 'Saldo_Final':101308*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFTO_FX_Exposure_Model_01.10.2024_-_With_Pnl___Tax_Credit.xlsx', 'Balance_Name':'Related parties - loan agreement', 'Currency_Denomination':'USD denominated', 'Class_Account_2':'Current Liabilities', 'file_sheet':'Balance Sheet', 'Saldo_Final':1658086*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFTO_FX_Exposure_Model_01.10.2024_-_With_Pnl___Tax_Credit.xlsx', 'Balance_Name':'Related parties - loan agreement', 'Currency_Denomination':'USD denominated', 'Class_Account_2':'Non-current assets', 'file_sheet':'Balance Sheet', 'Saldo_Final':14511*10**3}, index=[n])])
n+=1
files_df = pd.concat([files_df, pd.DataFrame({'file_name':'BRFTO_FX_Exposure_Model_01.10.2024_-_With_Pnl___Tax_Credit.xlsx', 'Balance_Name':'Related parties - loan agreement', 'Currency_Denomination':'BRL, out of OCP', 'Class_Account_2':'Non-current assets', 'file_sheet':'Balance Sheet', 'Saldo_Final':4168929*10**3}, index=[n])])

files_df.Class_Account_2 = files_df.Class_Account_2.str.lower()

# col with query
cols_list = ['file_name','Balance_Name','Currency_Denomination','Class_Account_2']
files_df['id'] = files_df.index
files_df_melt = pd.melt(files_df, id_vars=['id'], value_vars=cols_list)
files_df_melt['query'] = files_df_melt.variable + ' == \'' + files_df_melt.value + '\' and '
files_df_query = files_df_melt.groupby(as_index=False, dropna=False, by=['id'])['query'].agg(lambda x: ' '.join(x))
files_df_query['query'] = files_df_query['query'].str[:-5] # del last 'and'
files_df_work = files_df.merge(files_df_query, on='id')
files_df_work

,file_name,Balance_Name,Currency_Denomination,Class_Account_2,file_sheet,Saldo_Final,id,query
0,BRSFT_FX_Exposure_Model_01.10.2024_-_With_Pnl_...,Related parties - loan agreement,"BRL, out of OCP",current liabilities,Balance Sheet,3.457110e+09,0,file_name == 'BRSFT_FX_Exposure_Model_01.10.20...
1,BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax...,Borrowings Third Parties,USD denominated,current liabilities,Balance Sheet,1.011800e+07,1,file_name == 'BRFHRG_FX_Exposure_01.10.2024_-_...
2,BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax...,Borrowings Third Parties,"BRL denominated, in OCP",current liabilities,Balance Sheet,2.999600e+07,2,file_name == 'BRFHRG_FX_Exposure_01.10.2024_-_...
3,BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax...,Borrowings Third Parties,USD denominated,non-current liabilities,Balance Sheet,1.622870e+08,3,file_name == 'BRFHRG_FX_Exposure_01.10.2024_-_...
4,BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax...,Borrowings Third Parties,"BRL denominated, in OCP",non-current liabilities,Balance Sheet,1.682190e+08,4,file_name == 'BRFHRG_FX_Exposure_01.10.2024_-_...
5,BRFHRG_FX_Exposure_01.10.2024_-_With_pnl___Tax...,Borrowings Third Parties,"BRL, out of OCP",non-current liabilities,Balance Sheet,1.013080e+08,5,file_name == 'BRFHRG_FX_Exposure_01.10.2024_-_...
6,BRFTO_FX_Exposure_Model_01.10.2024_-_With_Pnl_...,Related parties - loan agreement,USD denominated,current liabilities,Balance Sheet,1.658086e+09,6,file_name == 'BRFTO_FX_Exposure_Model_01.10.20...
7,BRFTO_FX_Exposure_Model_01.10.2024_-_With_Pnl_...,Related parties - loan agreement,USD denominated,non-current assets,Balance Sheet,1.451100e+07,7,file_name == 'BRFTO_FX_Exposure_Model_01.10.20...
8,BRFTO_FX_Exposure_Model_01.10.2024_-_With_Pnl_...,Related parties - loan agreement,"BRL, out of OCP",non-current assets,Balance Sheet,4.168929e+09,8,file_name == 'BRFTO_FX_Exposure_Model_01.10.20...


In [223]:
# read excel files into 1 df
files_data = pd.DataFrame()
for file in os.listdir('Map'):
    if file == 'BRSFT_FX_Exposure_Model_01.10.2024_-_With_Pnl___Tax_Credit.xlsx':
        file_data = pd.read_excel(f'Map\\{file}', sheet_name = 'Balance Sheet', usecols='A:G', header=1)
    else:
        file_data = pd.read_excel(f'Map\\{file}', sheet_name = 'Balance Sheet', usecols='A:G')
    file_data['file_name'] = file 
    files_data = pd.concat([files_data, file_data])
assert len(files_data) >= 197+568+234

files_data_work = files_data.loc[~files_data['Balance Name'].isna(), ['file_name', 'Seg Conta', 'Saldo Final','Currency Denomination','Balance Name','Class Account','Class Account 2']].rename(columns={'Seg Conta':'Seg_Conta', 'Saldo Final':'Saldo_Final','Currency Denomination':'Currency_Denomination','Balance Name':'Balance_Name','Class Account':'Class_Account','Class Account 2':'Class_Account_2'})
files_data_work.Class_Account_2 = files_data_work.Class_Account_2.str.lower()

# choose info only by conditions
conditions_data = pd.DataFrame()
for i in files_df.index:
    condition = files_df_work.loc[i,'query']
    condition_df = files_data_work.query(condition)
    conditions_data = pd.concat([conditions_data, condition_df])
# check if conditions is equal right sum
conditions_data_pivot = conditions_data.groupby(as_index=False, dropna=False, by=cols_list)['Saldo_Final'].agg('sum') # group conditions data 
check_merge = files_df.merge(conditions_data_pivot, on=cols_list) # merge with manual conditions df
check_merge[['Saldo_Final_x','Saldo_Final_y']] = check_merge[['Saldo_Final_x','Saldo_Final_y']].astype(int)
check_merge = is_approximately_equal_for_cols(check_merge, 'Saldo_Final_x','Saldo_Final_y', tolerance=0.01)
assert check_merge['Diff__more_0.01'].unique().tolist() == [False] # check if sums is equal

# merge full cover on excels 
full_cover_to_seg_conta_merge = data.merge(conditions_data, left_on='glAccount', right_on='Seg_Conta', how='outer').sort_values('Seg_Conta')
# merge both part with ARAP on excels
full_data_merge = full_data_merge.merge(conditions_data, left_on='glAccount', right_on='Seg_Conta', how='outer').sort_values('glAccount')

In [234]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_BRL_ARAP_full_cover.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        pd.DataFrame({'SQL':query}, index=[1]).to_excel(writer, index=False, sheet_name='SQL')
        data.to_excel(writer, index=False, sheet_name='full_cover')
        data_ARAP.to_excel(writer, index=False, sheet_name='ARAP')
        full_data_merge.to_excel(writer, index=False, sheet_name='merge')
        EN_df_work.to_excel(writer, index=False, sheet_name='EN_desc')
        full_cover_to_seg_conta_merge.to_excel(writer, index=False, sheet_name='map_check')